In [1]:
import json
import os
import pandas as pd
import pymysql

In [32]:
def read_atributes(file):

    atributos = {}
#     remove = ['nFLows']

    with open(file, 'r') as signature_file:
        signature = json.load(signature_file)
        cardinalidade = signature['metadata']['cardinality']
        for step in signature['signature']:
            for atribute in signature['signature'][step]:
#                 if atribute not in remove:
                atributos.setdefault(atribute, [signature['signature'][step][atribute]['value_min'],signature['signature'][step][atribute]['value_max']])

    return atributos, cardinalidade

In [33]:
sig_path = '/home/amanda/Downloads/TCC/Assinaturas_sanitizadas/scanvuln/'

In [34]:
files = [os.path.join(sig_path, f) for f in os.listdir(sig_path) if os.path.isfile(os.path.join(sig_path, f))]

In [38]:
def query(atributos, cardinalidade, tabela):
    contar = ['nPortSrc', 'nPortDst', 'nAddrSrc', 'nAddrDst', 'nProto', 'nFLows']
    remove = ['nFLows']
    flags = {'all_have_tcp_A': 16, 
             'all_have_tcp_R': 4, 
             'all_have_tcp_S': 2, 
             'all_have_tcp_F': 1
            }
    dic = {'srcport': 'L4_SRC_PORT',
            'nPortSrc': 'L4_SRC_PORT',
            'destport': 'L4_DST_PORT',
            'nPortDst': 'L4_DST_PORT',
            'nAddrSrc': 'IPV4_SRC_ADDR',
            'nAddrDst': 'IPV4_DST_ADDR',
            'proto': 'PROTOCOL',
            'nProto': 'PROTOCOL',
            'nBytesSrc': 'IN_BYTES',
            'nBytesDst': 'OUT_BYTES',
            'nPcktsSrc': 'IN_PKTS',
            'nPcktsDst': 'OUT_PKTS'
            }
    
    for atrib_name, atrib_value in atributos.items():
        if atrib_name in remove:
            prefixo = 'select count(*) as nFLows'
        else:
            prefixo = 'select count(*)'
    
    for atrib_name, atrib_value in atributos.items():
        if atrib_name in contar and atrib_name not in remove:
            prefixo += ', count(distinct ' + dic[atrib_name] + ') as ' + str(atrib_name)
        
    sql = prefixo + ' from ' + tabela + ' where '
    statement = ''
    cont = 0
    queries = []
    
    for atrib_name, atrib_value in atributos.items():
        if atrib_name not in contar and atrib_name not in flags:
            if cont != 0:
                statement += ' AND '
            statement += '(' + dic[atrib_name] + ' between ' + str(atrib_value[0]) + ' AND ' + str(atrib_value[1]) + ')'
            
        cont = cont + 1
    
    soma_flags = 0
    for atrib_name, atrib_value in atributos.items():
        if atrib_name in flags:
            soma_flags += flags[atrib_name]
    
    # Se soma_flags for diferente de zero
    if soma_flags:
        statement += ' AND ((TCP_FLAGS = ' + str(soma_flags) + ') OR (TCP_FLAGS = ' + str(soma_flags+8) + ') OR (TCP_FLAGS = ' + str(soma_flags+32) + ') OR (TCP_FLAGS = ' + str(soma_flags+8+32) + '))'
            
    minimo = 1
    for interval in range(1000,51000,1000):
        if cardinalidade == '1-N':
            sufixo = ' AND (idx between ' + str(minimo) + ' AND ' + str(interval) + ') group by IPV4_SRC_ADDR;'
        elif cardinalidade == 'N-1':
            sufixo = ' AND (idx between ' + str(minimo) + ' AND ' + str(interval) + ') group by IPV4_DST_ADDR;'
        else:
            sufixo = ' AND (idx between ' + str(minimo) + ' AND ' + str(interval) + ') group by IPV4_SRC_ADDR, IPV4_DST_ADDR;'
        minimo = interval+1
        queries.append(sql + statement + sufixo)
        print(sql + statement + sufixo)

    return queries

In [39]:
def consultar(query):
    db = pymysql.connect("200.145.216.190","root","1nstalation","ntopng")
    cursor = db.cursor()
    try:
        cursor.execute(query)
        columns = [column[0] for column in cursor.description]
        results = cursor.fetchall()
        db.close()
        return columns, results
    except:
        print ("Error: unable to fecth data")
        return [], ()
        db.close()

In [37]:
queries = []
vp = 0
vn = 0
fp = 0
fn = 0

total = 50 * len(files) * 2

for file in files:
    print('\n\nAssinatura: ' + str(file))
    atributos, cardinalidade = read_atributes(file)
    queries = query(atributos, cardinalidade, 'scanvuln_flows')
    
    for sql in queries:
        colunas, resultados = consultar(sql)
        df = pd.DataFrame(list(resultados), columns=colunas)
        classificador = 1 # É um verdadeiro positivo
        for coluna in colunas:
            for valor in df[coluna].values:
                if not(atributos[coluna][0] <= valor <= atributos[coluna][1]):
                    # É um falso negativo
                    classificador = 0
        
        if (classificador):
            vp = vp + 1
        else:
            fn = fn + 1
    
    queries = query(atributos, cardinalidade, 'leg_flows')
    
    for sql in queries:
        colunas, resultados = consultar(sql)
        df = pd.DataFrame(list(resultados), columns=colunas)
        classificador = 1 # É um verdadeiro negativo
        for coluna in colunas:
            for valor in df[coluna].values:
                if atributos[coluna][0] <= valor <= atributos[coluna][1]:
                    # É um falso positivo
                    classificador = 0
                    
        if (classificador):
            vn = vn + 1
        else:
            fp = fp + 1
        
    
print('\nVerdadeiro Positivo: ' + str(vp))
print('\nFalso Negativo: ' + str(fn))
print('\nFalso Positivo: ' + str(fp))
print('\nVerdadeiro Negativo: ' + str(vn))



Assinatura: /home/amanda/Downloads/TCC/Assinaturas_sanitizadas/scanvuln/scanvuln3.sig
select count(*) as nFLows from scanvuln_flows where (L4_DST_PORT between 80 AND 80) AND (PROTOCOL between 6 AND 6) AND ((TCP_FLAGS = 22) OR (TCP_FLAGS = 30) OR (TCP_FLAGS = 54) OR (TCP_FLAGS = 62)) AND (idx between 1 AND 1000) group by IPV4_SRC_ADDR;
select count(*) as nFLows from scanvuln_flows where (L4_DST_PORT between 80 AND 80) AND (PROTOCOL between 6 AND 6) AND ((TCP_FLAGS = 22) OR (TCP_FLAGS = 30) OR (TCP_FLAGS = 54) OR (TCP_FLAGS = 62)) AND (idx between 1001 AND 2000) group by IPV4_SRC_ADDR;
select count(*) as nFLows from scanvuln_flows where (L4_DST_PORT between 80 AND 80) AND (PROTOCOL between 6 AND 6) AND ((TCP_FLAGS = 22) OR (TCP_FLAGS = 30) OR (TCP_FLAGS = 54) OR (TCP_FLAGS = 62)) AND (idx between 2001 AND 3000) group by IPV4_SRC_ADDR;
select count(*) as nFLows from scanvuln_flows where (L4_DST_PORT between 80 AND 80) AND (PROTOCOL between 6 AND 6) AND ((TCP_FLAGS = 22) OR (TCP_FLAGS = 

KeyError: 'nFLows'

In [41]:
for file in files:
    print('\n\nAssinatura: ' + str(file))
    atributos, cardinalidade = read_atributes(file)
    queries = query(atributos, cardinalidade, 'scanvuln_flows')
    
    for sql in queries:
        colunas, resultados = consultar(sql)
        df = pd.DataFrame(list(resultados), columns=colunas)
        print(df)



Assinatura: /home/amanda/Downloads/TCC/Assinaturas_sanitizadas/scanvuln/scanvuln3.sig
select count(*) from scanvuln_flows where (L4_DST_PORT between 80 AND 80) AND (PROTOCOL between 6 AND 6) AND ((TCP_FLAGS = 22) OR (TCP_FLAGS = 30) OR (TCP_FLAGS = 54) OR (TCP_FLAGS = 62)) AND (idx between 1 AND 1000) group by IPV4_SRC_ADDR;
select count(*) from scanvuln_flows where (L4_DST_PORT between 80 AND 80) AND (PROTOCOL between 6 AND 6) AND ((TCP_FLAGS = 22) OR (TCP_FLAGS = 30) OR (TCP_FLAGS = 54) OR (TCP_FLAGS = 62)) AND (idx between 1001 AND 2000) group by IPV4_SRC_ADDR;
select count(*) from scanvuln_flows where (L4_DST_PORT between 80 AND 80) AND (PROTOCOL between 6 AND 6) AND ((TCP_FLAGS = 22) OR (TCP_FLAGS = 30) OR (TCP_FLAGS = 54) OR (TCP_FLAGS = 62)) AND (idx between 2001 AND 3000) group by IPV4_SRC_ADDR;
select count(*) from scanvuln_flows where (L4_DST_PORT between 80 AND 80) AND (PROTOCOL between 6 AND 6) AND ((TCP_FLAGS = 22) OR (TCP_FLAGS = 30) OR (TCP_FLAGS = 54) OR (TCP_FLAGS = 

Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)

Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)

Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)

Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)

Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)

Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []
Empty DataFrame
Columns: [count(*)]
Index: []


KeyboardInterrupt: 